## Formación en NLP e interpretación de textos + extracción de contenido ##

*Introducción*

En pocas ocasiones, cuando una empresa recibe una solicitud de presupuesto, éste incluye las
referencias de producto exactas. En cambio, los clientes solicitan los presupuestos por ejemplo
en un e-mail:
Hola, quisiera cuatro cajas de arroz, seis pallets de azúcar [...]

*Ejercicio*

El sistema a desarrollar consiste en crear un modelo que pueda identificar qué productos se
solicitan.
Para ello, te proporcionamos el siguiente dataset:
https://apioverstand.es/training/dataset_productos.csv

Con ellos, genera manualmente varios textos de muestra donde se soliciten varios de estos
productos. La idea es que tu sistema traduzca, por ejemplo en el caso anterior:

- cien chuletas de cordero paletilla > producto id 7 | 100 unidades

- 15Aguardiente de orujo El Afilador 70 cl > producto id 63 15 unidades

**PASO 1. Importar librerias**

In [1]:
# IMPORTAMOS LIBRERIAS NECESARIAS
import pandas as pd
# pip install unidecode
from unidecode import unidecode

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

**PASO 2. Cargar dataset**

In [2]:
# Cargar del dataset en un dataframe 
df_productos = pd.read_csv("dataset_productos.csv")

# Mostrar 5 filas del DataFrame para verificar la carga exitosa
df_productos.head(5)

,name,ID
0,Bebida láctea sin lactosa de fresa Hacendado,1
1,Ginebra 15 botanicals Blumara,2
2,Queso fresco batido desnatado Hacendado 0% mat...,3
3,Yogur bífidus desnatado con lima y limón Danon...,4
4,Helado After Dinner Magnum sin gluten 10 ud.,5


**PASO 3. Buscar registros nulos o vacios**

In [3]:
# Mostrar registros con datos nulos o vacíos
df_nulls = df_productos[df_productos.isnull().any(axis=1)]
df_nulls

,name,ID
29,NaN,30


**PASO 4. Eliminar registros nulos**

Eliminamos cualquier registro que no contenga producto, ya que no tiene sentido tener una ID (en este caso la id30) que no esta asociada a nada.

In [4]:
# Eliminamos registros vacios, ya que no tiene sentido manener una ID que no esta asociada a nada
# y volvemos a comprobar si existen registros en el df
df_productos = df_productos.dropna(subset=['name'])
df_nulls = df_productos[df_productos.isnull().any(axis=1)]
df_nulls

,name,ID


**PASO 5. Buscar registros duplicados**

In [5]:
# Buscaremos registros duplicados
df_duplicates = df_productos[df_productos.duplicated(['name', 'ID'], keep=False)]
df_duplicates

,name,ID


**PASO 6. Eliminar todas las tildes de los productos**

Eliminamos todas las tildes de los productos para facilitar la busqueda, ya que, la mayoria de usuarios cuando escriben suelen hacerlo sin tilde. Aun asi, de la forma que esta realizado el codigo, con tilde o sin tilde, el codigo identifica correctamente el producto.

In [6]:
# Para evitar posteriores confusiones, eliminaremos todas las tildes de todos los productos
df_productos['name'] = df_productos['name'].apply(lambda x: unidecode(str(x)))
df_productos.reset_index(drop=True, inplace=True)
df_productos.head(5)

,name,ID
0,Bebida lactea sin lactosa de fresa Hacendado,1
1,Ginebra 15 botanicals Blumara,2
2,Queso fresco batido desnatado Hacendado 0% mat...,3
3,Yogur bifidus desnatado con lima y limon Danon...,4
4,Helado After Dinner Magnum sin gluten 10 ud.,5


**PASO 7. Obtencion de la *similitud del coseno* entre una frase y un registro determinado**

La busqueda del producto la haremos comparando el texto que el usuario escribe con el texto contenido en todos los registros. La comparacion no la haremos letra por letra sino que aplicaremos la similitud del coseno a la frase escrita por el usuario con respecto a todos y cada uno de los productos. De esta forma, obtendremos el producto que mayor porcentaje de similitud tenga. Esto permite que aunque el usuario cometa alguna falta de ortografia, o incluso, omita palabras, el buscador seguira identificando el producto correctamente.
Para ello utilizaremos funciones como las que siguen.

In [7]:
# FUNCION QUE DEVUELVE EL PORCENTAJE DE SIMILITUD ENTRE UNA FRASE Y UN REGISTRO DETERMINADO
def porcentaje_similitud_registro(indice_registro, frase):
    descripcion_producto = df_productos.loc[indice_registro, 'name']
    vectorizer = CountVectorizer()
    matriz_conteo = vectorizer.fit_transform([frase, descripcion_producto])
    similitud = cosine_similarity(matriz_conteo[0], matriz_conteo[1])[0, 0]    
    return similitud * 100

frase_de_ejemplo = porcentaje_similitud_registro(0,'Bebida láctea sin lactosa de fresa Hacendado') 
frase_de_ejemplo

85.7142857142857

**PASO 8. Localizacion del producto y su correspondiente ID**

En esta funcion, y valiendonos de la funcion anterior, haremos la comparacion entre la frase escrita por el usuario y todos y cada uno de los productos del dataset. Aquel que mayor porcentaje de similitud ofrezca, sera el articulo seleccionado.

In [8]:
# FUNCION QUE NOS DEVUELVE EL REG. MAS PARECIDO, SU ID, Y EL PORCENTAJE DE SIMILITUD
# DE UNA FRASE ENVIADA COMO PARAMETRO
def registro_mas_similar(frase):
    porcentaje_similitud_mas_alto = 0
    registro_mas_similar = None
    id_mas_similar = None

    for indice_registro in range(len(df_productos)):
        porcentaje_similitud = porcentaje_similitud_registro(indice_registro, frase)

        if porcentaje_similitud > porcentaje_similitud_mas_alto:
            porcentaje_similitud_mas_alto = round(porcentaje_similitud,2)
            registro_mas_similar = indice_registro
            id_mas_similar = df_productos.loc[indice_registro, 'ID']

    return registro_mas_similar, id_mas_similar, porcentaje_similitud_mas_alto

# FRASES DE EJEMPLO
# frase = "cien chuletas de cordero paletilla"
frase = "setenta y ochoBebida de arroz con almendras ecológica Isola Bio sin gluten brik 250 ml"

frase_de_ejemplo = frase
registro, id_registro, porcentaje = registro_mas_similar(frase)
articulo = df_productos.iloc[registro][0]

print('Pedido:', frase)
print('Articulo:', articulo)
print('ID del articulo:', id_registro)
print('Similitud:', porcentaje, '%')

Pedido: setenta y ochoBebida de arroz con almendras ecológica Isola Bio sin gluten brik 250 ml
Articulo: Bebida de arroz con almendras ecologica Isola Bio sin gluten brik 250 ml.
ID del articulo: 1686
Similitud: 81.54 %


**PASO 9. Obtencion de la cantidad**

Para determinar la cantidad, deducimos que siempre se antepone al producto, ya sea con texto o con numero. Por tanto, extraemos el texto escrito por el usuario que no coincide con el producto. Ese texto extra se deduce que es la cantidad o alguna peticion sobre el producto. Lo haremos comparando la frase del usuario con el articulo. Aquel texto que no se encuentre en el articulo pero si en la frase del usuario, sera texto extra que contendra muy probablemente la cantidad solicitada.

In [9]:
def obtener_texto_extra(articulo, frase):
    # Eliminar el punto si es el último caracter de la cadena
    if articulo.endswith('.'):
        articulo = articulo[:-1]             

    # Calcular la longitud del texto extra
    diferencia_longitud = len(frase) - len(articulo)
    texto_extra = frase[:diferencia_longitud]

    return texto_extra

texto_extra = obtener_texto_extra(articulo, frase)
print('Frase original:', frase)
print('Articulo:', articulo)
print('Texto extra:', texto_extra)

Frase original: setenta y ochoBebida de arroz con almendras ecológica Isola Bio sin gluten brik 250 ml
Articulo: Bebida de arroz con almendras ecologica Isola Bio sin gluten brik 250 ml.
Texto extra: setenta y ocho


**PASO 10. Conversion de la cantidad de formato texto a formato numerico**

En algunos casos, hay usuarios que en lugar de escribir la cantidad en numero lo hacen en letra. Por tanto, haremos un codigo que, en caso de que el usuario indique la cantidad con texto, sea capaz de convertir ese texto a su equivalente numerico. Para ello usaremos un codigo, con un diccionario, que aunque no tiene explícitamente todas las palabras numéricas, está diseñado para analizar ciertos patrones comunes en el lenguaje y convertirlos en sus equivalentes numéricos.

In [10]:
def convertir_numero_texto_a_numero(texto):
    numeros_texto = {
        'cero': 0, 'un': 1, 'una': 1, 'uno': 1, 'dos': 2, 'tres': 3, 'cuatro': 4,
        'cinco': 5, 'seis': 6, 'siete': 7, 'ocho': 8, 'nueve': 9, 'diez': 10,
        'once': 11, 'doce': 12, 'trece': 13, 'catorce': 14, 'quince': 15,
        'dieciseis': 16, 'diecisiete': 17, 'dieciocho': 18, 'diecinueve': 19,
        'veinte': 20, 'veintiuno': 21, 'veintidos': 22, 'veintitres': 23,
        'veinticuatro': 24, 'veinticinco': 25, 'veintiseis': 26, 'veintisiete': 27,
        'veintiocho': 28, 'veintinueve': 29, 'treinta': 30, 'cuarenta': 40,
        'cincuenta': 50, 'sesenta': 60, 'setenta': 70, 'ochenta': 80, 'noventa': 90,
        'cien': 100, 'ciento': 100, 'ciento uno': 101, 'doscientos': 200,
        'trescientos': 300, 'cuatrocientos': 400, 'quinientos': 500,
        'seiscientos': 600, 'setecientos': 700, 'ochocientos': 800,
        'novecientos': 900, 'mil': 1000, 'millon': 1000000, 'millones': 1000000,
    }

    # Dividir el texto en palabras
    palabras = texto.lower().split()
    numero = 0
    i = 0

    # Procesar cada palabra
    while i < len(palabras):
        palabra_actual = palabras[i]

        # Revisar si la palabra actual está en el diccionario de números en texto
        if palabra_actual in numeros_texto:
            numero += numeros_texto[palabra_actual]
            i += 1
        else:
            i += 1

    return numero if numero > 0 else None  # Devolver el número, si es posible

# ESCRIBA CUALQUIER NUMERO CON LETRAS PARA PROBAR LA FUNCION
texto_numero = "trescientos cuarenta y nueve"
numero = convertir_numero_texto_a_numero(texto_numero)
print(f"Texto de prueba: {texto_numero}\nNúmero: {numero}")

Texto de prueba: trescientos cuarenta y nueve
Número: 349


**PASO 11. Obtencion de la cantidad final**

Una vez ya tenemos el articulo localizado y el texto extra donde viene la cantidad. Analizamos dicho texto extra, que no pertenece al articulo y, usando las dos funciones anteriores extraemos la cantidad de forma numerica. Si el usuario ya la escribio de forma numerica hace la conversion directamente, pero si lo hizo en forma de texto, usara la funcion del paso anterior *convertir_numero_texto_a_numero*

El resultado final sera una variable con la cantidad solicitada en formato numero entero.

In [11]:
def obtener_cantidad(texto_extra):
    try:
        cantidad = int(texto_extra)
    except ValueError:
        cantidad = convertir_numero_texto_a_numero(texto_extra)
    return cantidad

# Ejemplo de uso
texto_extra = "doscientos quince"
cantidad = obtener_cantidad(texto_extra)
print(f"Cantidad: {cantidad}")

Cantidad: 215


**COMPROBACION DEL RESULTADO FINAL**

Una vez tenemos los datos identificados, tanto el articulo, como la cantidad numerica, podemos mostrar el resultado final.

Habilite cualquiera de los siguientes ejemplos y compruebe el resultado.

In [12]:
# pedido_del_cliente = '15Aguardiente de orujo El Afilador 70 cl'
pedido_del_cliente = 'cien chuletas de cordero'
# pedido_del_cliente = 'ciento veinte uva negra sin semillas'
# pedido_del_cliente = 'cienchuletas de cordero paletilla'
# pedido_del_cliente = 'diez ISABEL mejillones en salsa vieira'
# pedido_del_cliente = '25 Horchata de chufa chufi original botella 1l'
# pedido_del_cliente = 'diecisiete mayonesa hacendado'

frase = pedido_del_cliente
registro, id_registro, porcentaje = registro_mas_similar(frase)
articulo = df_productos.iloc[registro][0]
texto_extra = obtener_texto_extra(articulo, frase)
cantidad = obtener_cantidad(texto_extra)

print('Pedido:', frase)
print("")
print('Articulo:', articulo)
print('ID:', id_registro)
print('Cantidad:', cantidad, 'unidades.')
print('[Similitud:', porcentaje, '%]')

Pedido: cien chuletas de cordero

Articulo: Chuletas de cordero paletilla
ID: 7
Cantidad: 100 unidades.
[Similitud: 75.0 %]
